In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import xarray as xr

%matplotlib inline

In [2]:
imin = 376 - 1  # want 380
imax = 470 # want 470
rim = 10
nx = np.arange(imin, imax) + 1
nxb = nx
for ii in range(rim-1):
    nxb = np.append(nxb, nx)
print(nxb.shape)

(950,)


In [3]:
filename = 'tracers/west/bioOBC_const.nc'
xds = xr.open_dataset('/results/nowcast-sys/'+filename, decode_times=False)
print (xds)

<xarray.Dataset>
Dimensions:       (deptht: 40, time_counter: 1, xbT: 870, yb: 1)
Coordinates:
  * deptht        (deptht) float32 0.5 1.5 2.50001 3.50003 4.50007 5.50015 ...
  * time_counter  (time_counter) float32 0.0
Dimensions without coordinates: xbT, yb
Data variables:
    nav_lat       (yb, xbT) float32 ...
    nav_lon       (yb, xbT) float32 ...
    nbidta        (yb, xbT) int32 ...
    nbjdta        (yb, xbT) int32 ...
    nbrdta        (yb, xbT) int32 ...
    NH4           (time_counter, deptht, yb, xbT) float32 ...
    DIA           (time_counter, deptht, yb, xbT) float32 ...
    CRY           (time_counter, deptht, yb, xbT) float32 ...
    MYRI          (time_counter, deptht, yb, xbT) float32 ...
    MICZ          (time_counter, deptht, yb, xbT) float32 ...
    PON           (time_counter, deptht, yb, xbT) float32 ...
    DON           (time_counter, deptht, yb, xbT) float32 ...
    bSi           (time_counter, deptht, yb, xbT) float32 ...
    O2            (time_counter, de

In [4]:
varis = ['NH4', 'DIA', 'CRY', 'MYRI', 'MICZ', 'PON', 'DON', 'bSi', 'O2', 'tur']

In [5]:
NH4 = xds.NH4[:]
for id in range(40):
    print(np.max(NH4[0,id,0,0]) - np.min(NH4[0, id, 0, 0]))

<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float32 0.5
    time_counter  float32 0.0
<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float32 1.5
    time_counter  float32 0.0
<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float32 2.50001
    time_counter  float32 0.0
<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float32 3.50003
    time_counter  float32 0.0
<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float32 4.50007
    time_counter  float32 0.0
<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float32 5.50015
    time_counter  float32 0.0
<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float32 6.50031
    time_counter  float32 0.0
<xarray.DataArray 'NH4' ()>
array(0.0, dtype=float32)
Coordinates:
    deptht        float3

In [6]:
for vari in varis:
    print (vari, np.max(xds[vari][:])-np.min(xds[vari][:]))

NH4 <xarray.DataArray 'NH4' ()>
array(0.9848267436027527)
DIA <xarray.DataArray 'DIA' ()>
array(0.0)
CRY <xarray.DataArray 'CRY' ()>
array(0.0)
MYRI <xarray.DataArray 'MYRI' ()>
array(0.0)
MICZ <xarray.DataArray 'MICZ' ()>
array(0.0)
PON <xarray.DataArray 'PON' ()>
array(0.0)
DON <xarray.DataArray 'DON' ()>
array(0.0)
bSi <xarray.DataArray 'bSi' ()>
array(0.0)
O2 <xarray.DataArray 'O2' ()>
array(0.0)
tur <xarray.DataArray 'tur' ()>
array(0.0)


In [7]:
print (xds['tur'].shape)

(1, 40, 1, 870)


In [8]:
# was running, fortran, 381 to 467
# want to run, 380 to 470 so add one to start and 3 to end

In [9]:
def pad(xds, vari, data, rim, nd=40):
    nz1 = np.ones((1, nd, 1, rim*(imax-imin)))
    for idd in range(nd):
        factor = float(xds[vari][0, idd, 0, 45])
        nz1[0, idd, 0, :] = nz1[0, idd, 0, :] * factor
    data[vari] = nz1
    return data

In [10]:
print(data['NH4'].shape)
print(nxb.shape)
print(xds.deptht.shape)
print(xds.time_counter)

NameError: name 'data' is not defined

In [11]:
data = {}
for vari in varis:
    data = pad(xds, vari, data, rim)

da = {}
for var in varis:
    print(var, data[var].shape)
    da[var] = xr.DataArray(
        data=data[var],
        name=var,
        dims=('time_counter', 'deptht', 'yb', 'xb'),
        coords={
            'time_counter': xds.time_counter,
            'deptht': xds.deptht,
            'yb': [0],
            'xb': nxb,
        },
        attrs=xds[var].attrs)

NH4 (1, 40, 1, 950)
DIA (1, 40, 1, 950)
CRY (1, 40, 1, 950)
MYRI (1, 40, 1, 950)
MICZ (1, 40, 1, 950)
PON (1, 40, 1, 950)
DON (1, 40, 1, 950)
bSi (1, 40, 1, 950)
O2 (1, 40, 1, 950)
tur (1, 40, 1, 950)


In [12]:
ds = xr.Dataset(data_vars=da,
               coords={'time_counter': xds.time_counter,
                         'deptht': xds.deptht,
                         'yb': [0], 'xb': nxb},
               attrs=xds.attrs)

ds.to_netcdf('bioOBC_const_long.nc', unlimited_dims=('time_counter'),)

print(ds)

<xarray.Dataset>
Dimensions:       (deptht: 40, time_counter: 1, xb: 950, yb: 1)
Coordinates:
  * yb            (yb) int64 0
  * xb            (xb) int64 376 377 378 379 380 381 382 383 384 385 386 387 ...
  * deptht        (deptht) float32 0.5 1.5 2.50001 3.50003 4.50007 5.50015 ...
  * time_counter  (time_counter) float32 0.0
Data variables:
    MYRI          (time_counter, deptht, yb, xb) float64 1e-08 1e-08 1e-08 ...
    MICZ          (time_counter, deptht, yb, xb) float64 1e-08 1e-08 1e-08 ...
    O2            (time_counter, deptht, yb, xb) float64 160.0 160.0 160.0 ...
    CRY           (time_counter, deptht, yb, xb) float64 1e-08 1e-08 1e-08 ...
    bSi           (time_counter, deptht, yb, xb) float64 7.747e-06 7.747e-06 ...
    DIA           (time_counter, deptht, yb, xb) float64 1e-08 1e-08 1e-08 ...
    DON           (time_counter, deptht, yb, xb) float64 4.676 4.676 4.676 ...
    NH4           (time_counter, deptht, yb, xb) float64 0.2405 0.2405 ...
    tur           (time_

In [13]:
print(ds.NH4[0, :, 0, 45])

<xarray.DataArray 'NH4' (deptht: 40)>
array([  2.404570e-01,   3.658873e-01,   3.802824e-01,   3.914739e-01,
         4.021020e-01,   4.723583e-01,   5.327533e-01,   5.766485e-01,
         6.028428e-01,   6.235945e-01,   6.900732e-01,   7.107338e-01,
         7.163171e-01,   7.180485e-01,   7.181028e-01,   7.933621e-01,
         9.218616e-01,   9.562190e-01,   9.848267e-01,   9.778820e-01,
         9.655148e-01,   9.037943e-01,   5.324652e-01,   2.799609e-01,
         1.999781e-01,   6.547938e-02,   2.000105e-02,   3.222063e-04,
         2.693543e-14,   1.058683e-24,   2.791703e-35,   0.000000e+00,
         0.000000e+00,   0.000000e+00,   0.000000e+00,   0.000000e+00,
         0.000000e+00,   0.000000e+00,   0.000000e+00,   0.000000e+00])
Coordinates:
    yb            int64 0
    xb            int64 421
  * deptht        (deptht) float32 0.5 1.5 2.50001 3.50003 4.50007 5.50015 ...
    time_counter  float32 0.0
Attributes:
    grid:       SalishSea2
    units:      muM
    long_name:  